# Spike sorting

In [1]:
reAnalyse = True
engine = "dask"
GPU_available = True
sorterFolder='kilosort4_output'
training_folder = 'sorting_analyzer_training'
fullAnalyzer_folder = 'sorting_analyzer_full'
whitenFirst = True

## Set up everything
You shouldn't have to change anything from here so you can keep that part folded

### Load modules

In [2]:
import spikeinterface.full as si
import numpy as np
import pickle

import submitit
from memory_profiler import memory_usage
import time
import shutil
import os
from matplotlib import pyplot as plt

import asyncio
import gc

from mbTools import mbTools
from ipyfilechooser import FileChooser
import ipywidgets as widgets
from IPython.display import display
from IPython import get_ipython
import IPython
import pickleshare

from dask_jobqueue import SLURMCluster
from distributed import Client

%matplotlib widget

ImportError: Numba needs NumPy 2.0 or less. Got NumPy 2.1.

### Define a few variables

In [ ]:
duration_extract = 2 #min

#### Structural (important for the process but no need to change)

In [ ]:
last_job=None
sorter='kilosort4'

### Define a few functions

In [ ]:
def magicretrieve(stored_var):
   # myvar will contain the variable previously stored with "%store test"
   myvar_filename = get_ipython().ipython_dir + '/profile_default/db/autorestore/' + stored_var
   with open(myvar_filename, 'rb') as f:
      return pickle.load(f)

def magicstore(stored_var, value):
   # myvar will contain the variable previously stored with "%store test"
   myvar_filename = get_ipython().ipython_dir + '/profile_default/db/autorestore/' + stored_var
   with open(myvar_filename, 'wb') as f:
      pickle.dump(value,f)      

In [ ]:
def update_my_expe_choice(chooser):
    currentFile_data = str(chooser.selected)
    currentFile_mnt = os.path.join('/mnt/data/ahay',os.path.split(currentFile_data)[1])
    if not currentFile_data.startswith('/crnldata/'):
        print('please make sure to select the file on crnldata')
        return
    # check if the file already exists on /mnt/
    if os.path.isfile(currentFile_mnt):
        print(f"{currentFile_mnt} already exists")
    else:
        print(f"there is no version of {currentFile_data} on /mnt/")
        shouldCopy = False # it took a while (20 min for a file of about 150Gb)
        if shouldCopy:
            print(f'it will be copied to {currentFile_mnt}')
            startTime = time.time()
            shutil.copyfile(currentFile_data, currentFile_mnt)
            print(f'the transfer is complete, it took {time.time()-startTime} seconds')
        else:
            print('it can be transfered from here by changing the shouldCopy parameter but probably best not to because it takes a while and uses massive ressources')

    magicstore('currentFile_data', currentFile_data)
    magicstore('currentFile_mnt', currentFile_mnt)


def selectData(currentFile):
    print(currentFile)
    if currentFile is not None:
        pathName, fileName = os.path.split(currentFile)
    else:
        pathName = '/crnldata/waking/audrey_hay/'
        fileName = ''
    fc = FileChooser(path=pathName, filename=fileName, filter_pattern='NP_spikes_*.raw', select_default=True, show_only_dirs = False, title = "<b>Select file on crnldata</b>")
    display(fc)

    # Register callback function
    fc.register_callback(update_my_expe_choice)


In [ ]:
async def runFunction(engine,funcName,*params,**engineParams):
    start_time = time.time()
    match engine:
        case "dask":
            print("dask")
            cluster = SLURMCluster(
                                **engineParams,
                                log_directory="si_dask_logs",
                                scheduler_options={"dashboard_address": ":8780"} #port 8787 already used by jupyter
                                )
            cluster.scale(1)
            client = Client(cluster)

            client.get_versions(check=True)
    
            from dask.distributed import PipInstall
            plugin = PipInstall(packages=["git+https://github.com/SpikeInterface/spikeinterface.git"], pip_options=["--upgrade"])
            client.register_plugin(plugin)
            plugin = PipInstall(packages=["git+https://github.com/SpikeInterface/probeinterface.git"], pip_options=["--upgrade"])
            client.register_plugin(plugin)
            plugin = PipInstall(packages=["git+https://github.com/SpikeInterface/spikeinterface-gui.git"], pip_options=["--upgrade"])
            client.register_plugin(plugin)
            plugin = PipInstall(packages=["kilosort"], pip_options=["--upgrade"])
            client.register_plugin(plugin)

            plugin = PipInstall(packages=["numpy"], pip_options=["--upgrade"])
            client.register_plugin(plugin)

            print(cluster.job_script()) 

            future = client.submit(funcName, *params)
            res = future.result()
            
            #recording_layers = preprocess_traces(raw_rec)
            print(f"job done in {time.time()- start_time} s")

            # Close cluster
            client.close()
            cluster.close()

            return res
        
        case "submitit":
            print("submitit")

            executor = submitit.AutoExecutor(folder=os.getcwd()+'/si_logs/')
            executor.update_parameters(**engineParams)

            job = executor.submit(funcName, *params)

            # print the ID of your job
            print("submit job" + str(job.job_id))  

            # await a single result
            await job.awaitable().results()
            print(f"job {job.job_id} completed in " + str(time.time()-start_time) + " seconds")

            return job.result()

        case _:
            print("no engine")

            return funcName(*params) 

In [ ]:
def GenerateDict(rec, probe, folder, **sorter_params):
    rec = rec.set_probe(probe)
    si.set_global_job_kwargs(n_jobs=40, progress_bar=True, chunk_duration="1s")
    sorting = si.run_sorter(sorter, rec, verbose=True, folder=folder, remove_existing_folder=True, **sorter_params)
    return sorting


In [ ]:
def compute_analyzer(sorting,rec,probe,folder,append=False):

    rec = rec.set_probe(probe)
    si.set_global_job_kwargs(n_jobs=40, progress_bar=True, chunk_duration="1s")
    
    if append:
        sorting_analyzer = si.load_sorting_analyzer(folder)
    else:
        sorting_analyzer = si.create_sorting_analyzer(sorting, rec, sparse=True, folder=folder,overwrite=True)

    sorting_analyzer.compute("random_spikes", method="uniform", max_spikes_per_unit=500)
    sorting_analyzer.compute("waveforms")
    sorting_analyzer.compute("templates")
    sorting_analyzer.compute("noise_levels")
    sorting_analyzer.compute("unit_locations", method="monopolar_triangulation")
    sorting_analyzer.compute("isi_histograms")
    sorting_analyzer.compute("correlograms") #, window_ms=100, bin_ms=5.
    sorting_analyzer.compute("principal_components", n_components=3, mode='by_channel_global', whiten=True)
    sorting_analyzer.compute("quality_metrics", metric_names=["snr", "firing_rate"])
    sorting_analyzer.compute("template_similarity")
    sorting_analyzer.compute("spike_amplitudes")


    #sorting_analyzer.save_as(folder=folder, format='binary_folder')

    #return sorting_analyzer

In [ ]:
def preprocess_traces(rec):
    # filter traces
    rec = rec.astype('float32')
    rec_filt = si.bandpass_filter(rec)
    display(rec_filt)

    # Remove bad channels
    try:
        bad_channel_ids, channel_labels = si.detect_bad_channels(rec_filt)
        print('bad_channel_ids', bad_channel_ids)
        rec_filt = rec_filt.remove_channels(bad_channel_ids)
    except Exception as error:
        print("could not remove bad channels because there was an error:")
        print(error)

    # Account for the slight delays in recordings due to the fact the 384 channels are only digitilized with 32 ADCs

    inter_sample_shift=[0., 0., 0.07692308, 0.07692308, 0.15384615, 0.15384615, 0.23076923, 0.23076923, 0.30769231, 0.30769231,
                         0.38461538, 0.38461538, 0.46153846, 0.46153846, 0.53846154, 0.53846154, 0.61538462, 0.61538462, 0.69230769,
                           0.69230769, 0.76923077, 0.76923077, 0.84615385, 0.84615385,
                           0., 0., 0.07692308, 0.07692308, 0.15384615, 0.15384615, 0.23076923, 0.23076923, 0.30769231, 0.30769231,
                         0.38461538, 0.38461538, 0.46153846, 0.46153846, 0.53846154, 0.53846154, 0.61538462, 0.61538462, 0.69230769,
                           0.69230769, 0.76923077, 0.76923077, 0.84615385, 0.84615385,
                           0., 0., 0.07692308, 0.07692308, 0.15384615, 0.15384615, 0.23076923, 0.23076923, 0.30769231, 0.30769231,
                         0.38461538, 0.38461538, 0.46153846, 0.46153846, 0.53846154, 0.53846154, 0.61538462, 0.61538462, 0.69230769,
                           0.69230769, 0.76923077, 0.76923077, 0.84615385, 0.84615385,
                           0., 0., 0.07692308, 0.07692308, 0.15384615, 0.15384615, 0.23076923, 0.23076923, 0.30769231, 0.30769231,
                         0.38461538, 0.38461538, 0.46153846, 0.46153846, 0.53846154, 0.53846154, 0.61538462, 0.61538462, 0.69230769,
                           0.69230769, 0.76923077, 0.76923077, 0.84615385, 0.84615385,
                           0., 0., 0.07692308, 0.07692308, 0.15384615, 0.15384615, 0.23076923, 0.23076923, 0.30769231, 0.30769231,
                         0.38461538, 0.38461538, 0.46153846, 0.46153846, 0.53846154, 0.53846154, 0.61538462, 0.61538462, 0.69230769,
                           0.69230769, 0.76923077, 0.76923077, 0.84615385, 0.84615385,
                           0., 0., 0.07692308, 0.07692308, 0.15384615, 0.15384615, 0.23076923, 0.23076923, 0.30769231, 0.30769231,
                         0.38461538, 0.38461538, 0.46153846, 0.46153846, 0.53846154, 0.53846154, 0.61538462, 0.61538462, 0.69230769,
                           0.69230769, 0.76923077, 0.76923077, 0.84615385, 0.84615385,
                           0., 0., 0.07692308, 0.07692308, 0.15384615, 0.15384615, 0.23076923, 0.23076923, 0.30769231, 0.30769231,
                         0.38461538, 0.38461538, 0.46153846, 0.46153846, 0.53846154, 0.53846154, 0.61538462, 0.61538462, 0.69230769,
                           0.69230769, 0.76923077, 0.76923077, 0.84615385, 0.84615385,
                           0., 0., 0.07692308, 0.07692308, 0.15384615, 0.15384615, 0.23076923, 0.23076923, 0.30769231, 0.30769231,
                         0.38461538, 0.38461538, 0.46153846, 0.46153846, 0.53846154, 0.53846154, 0.61538462, 0.61538462, 0.69230769,
                           0.69230769, 0.76923077, 0.76923077, 0.84615385, 0.84615385,
                           0., 0., 0.07692308, 0.07692308, 0.15384615, 0.15384615, 0.23076923, 0.23076923, 0.30769231, 0.30769231,
                         0.38461538, 0.38461538, 0.46153846, 0.46153846, 0.53846154, 0.53846154, 0.61538462, 0.61538462, 0.69230769,
                           0.69230769, 0.76923077, 0.76923077, 0.84615385, 0.84615385,
                           0., 0., 0.07692308, 0.07692308, 0.15384615, 0.15384615, 0.23076923, 0.23076923, 0.30769231, 0.30769231,
                         0.38461538, 0.38461538, 0.46153846, 0.46153846, 0.53846154, 0.53846154, 0.61538462, 0.61538462, 0.69230769,
                           0.69230769, 0.76923077, 0.76923077, 0.84615385, 0.84615385,
                           0., 0., 0.07692308, 0.07692308, 0.15384615, 0.15384615, 0.23076923, 0.23076923, 0.30769231, 0.30769231,
                         0.38461538, 0.38461538, 0.46153846, 0.46153846, 0.53846154, 0.53846154, 0.61538462, 0.61538462, 0.69230769,
                           0.69230769, 0.76923077, 0.76923077, 0.84615385, 0.84615385,
                           0., 0., 0.07692308, 0.07692308, 0.15384615, 0.15384615, 0.23076923, 0.23076923, 0.30769231, 0.30769231,
                         0.38461538, 0.38461538, 0.46153846, 0.46153846, 0.53846154, 0.53846154, 0.61538462, 0.61538462, 0.69230769,
                           0.69230769, 0.76923077, 0.76923077, 0.84615385, 0.84615385,
                           0., 0., 0.07692308, 0.07692308, 0.15384615, 0.15384615, 0.23076923, 0.23076923, 0.30769231, 0.30769231,
                         0.38461538, 0.38461538, 0.46153846, 0.46153846, 0.53846154, 0.53846154, 0.61538462, 0.61538462, 0.69230769,
                           0.69230769, 0.76923077, 0.76923077, 0.84615385, 0.84615385,
                           0., 0., 0.07692308, 0.07692308, 0.15384615, 0.15384615, 0.23076923, 0.23076923, 0.30769231, 0.30769231,
                         0.38461538, 0.38461538, 0.46153846, 0.46153846, 0.53846154, 0.53846154, 0.61538462, 0.61538462, 0.69230769,
                           0.69230769, 0.76923077, 0.76923077, 0.84615385, 0.84615385,
                           0., 0., 0.07692308, 0.07692308, 0.15384615, 0.15384615, 0.23076923, 0.23076923, 0.30769231, 0.30769231,
                         0.38461538, 0.38461538, 0.46153846, 0.46153846, 0.53846154, 0.53846154, 0.61538462, 0.61538462, 0.69230769,
                           0.69230769, 0.76923077, 0.76923077, 0.84615385, 0.84615385,
                           0., 0., 0.07692308, 0.07692308, 0.15384615, 0.15384615, 0.23076923, 0.23076923, 0.30769231, 0.30769231,
                         0.38461538, 0.38461538, 0.46153846, 0.46153846, 0.53846154, 0.53846154, 0.61538462, 0.61538462, 0.69230769,
                           0.69230769, 0.76923077, 0.76923077, 0.84615385, 0.84615385,
                               ]
    rec_filt_shifted = si.phase_shift(rec_filt, inter_sample_shift=inter_sample_shift)

    # Remove the common noisy events (artefacts)
    rec_filt_ref = si.common_reference(rec_filt_shifted)
    display(rec_filt_ref)

    recording_layers = dict(
        raw = rec,
        filter = rec_filt,
        realigned = rec_filt_shifted,
        cmr = rec_filt_ref,
    )
    #si.plot_traces(recording_layers, backend='ipywidgets') #, mode='line'

    return recording_layers
    

In [ ]:
def check_drift(rec, probe):
    import numpy as np
      
    rec = rec.set_probe(probe)

    job_kwargs = dict(n_jobs=40, progress_bar=True, chunk_duration="1s")
    si.set_global_job_kwargs(**job_kwargs)
    
    recording_corrected, motion, motion_info = si.correct_motion(
            rec, preset="dredge_fast", folder=None, output_motion=True, output_motion_info=True#, **job_kwargs
        )
    return (recording_corrected, motion, motion_info)

In [ ]:
def whiten(rec,recording_layers,layerName):

    rec_whitened = si.WhitenRecording(rec)

    recording_layers[layerName] = rec_whitened

    return recording_layers

In [ ]:
def checkRessources():
    # check node and CPU information
    print("### Node counts: \nA: currently in use \B available")
    !sinfo -o%A
    print("### CPU counts: \nA: core currently in use \nI: available \nO: unavailable (maintenance, down, etc) \nT: total")
    !sinfo -o%C
    !sinfo

    # check some stats of our last job
    if last_job is not None:
        print('### CPU time and MaxRSS of our last job (about 1000Mb should be added to your MaxRSS (Mb) in order to cover safely the memory needs of the python runtime)###')
        os.system(f'sacct -j {last_job.job_id} --format="CPUTime,MaxRSS"')

## Choose data

In [ ]:

#currentFile_data = '/crnldata/waking/audrey_hay/NPX/NPX1/VB/Expe_2024-07-22_17-55-16/NP_spikes_2024-07-22T17_55_16.raw'
#mbTools.magicstore('currentFile_data', currentFile_data)
currentFile_data = magicretrieve('currentFile_data')
selectData(currentFile_data)

### Move data to /mnt/ if appropriate

In [ ]:

shouldCopy = False # it took a while (20 min for a file of about 150Gb)
if shouldCopy:
    currentFile_data = magicretrieve('currentFile_data')
    currentFile_mnt = magicretrieve('currentFile_mnt')
    print(f'The file {currentFile_data} will be copied to {currentFile_mnt}')
    startTime = time.time()
    shutil.copyfile(currentFile_data, currentFile_mnt)
    print(f'the transfer is complete, it took {time.time()-startTime} seconds')
else:
    print('it can be transfered from here by changing the shouldCopy parameter but probably best not to because it takes a while and uses massive ressources')


### Lazy load data and probe

In [ ]:
currentFile_data = magicretrieve('currentFile_data')
currentFile_mnt = magicretrieve('currentFile_mnt')

raw_rec = si.read_binary(currentFile_mnt, dtype='uint16', num_channels=384, sampling_frequency=30_000.,gain_to_uV=-1)
raw_rec.annotate(raw_path = currentFile_data)

In [ ]:
with open('/crnldata/waking/audrey_hay/NPX/NPXprobe.pkl', 'rb') as outp: 
    probe = pickle.load(outp)
probe.set_device_channel_indices(np.arange(384))

raw_rec = raw_rec.set_probe(probe)
display(si.plot_probe_map(raw_rec, with_channel_ids=True))

## PreProcessing

### Filter and apply common ref

In [ ]:
if engine=="dask":
    # takes about 15s
    cluster = SLURMCluster(
                        queue='CPU',
                        cores=1,
                        memory="3GB",
                        walltime="00:02:00",
                        log_directory="si_dask_logs",
                        scheduler_options={"dashboard_address": ":8780"} #port 8787 already used by jupyter
                        )
    cluster.scale(1)
    client = Client(cluster)


    print(cluster.job_script()) 

    start_time = time.time()
    future = client.submit(preprocess_traces, raw_rec)
    recording_layers = future.result() 


    #recording_layers = preprocess_traces(raw_rec)
    print(f"job done in {time.time()- start_time} s")

    # Close cluster
    client.close()
    cluster.close()

    si.plot_traces(recording_layers, backend='ipywidgets') #, mode='line'

In [ ]:
if engine=="submitit":
    # takes less than 10s
    start_time = time.time()

    executor = submitit.AutoExecutor(folder=os.getcwd()+'/si_logs/')
    executor.update_parameters(mem_gb=3, timeout_min=2, slurm_partition="CPU", cpus_per_task=4)
    job = executor.submit(preprocess_traces, raw_rec)

    # print the ID of your job
    print("submit job" + str(job.job_id))  

    # await a single result
    await job.awaitable().results()
    print(f"job {job.job_id} completed in " + str(time.time()-start_time) + " seconds")

    last_job = job
    recording_layers = job.result()

    si.plot_traces(recording_layers, backend='ipywidgets') #, mode='line'

### Correct for motion

In [ ]:
canal_lim=300 #limitation of canals to keep for drift correction
#TODO : allow drift calculation on a few channels only

if whitenFirst:
    recording_layers = whiten(recording_layers["cmr"],recording_layers,"whitenFirst")
    rec_2_correct = recording_layers["whitenFirst"]#.frame_slice(0,1000)#.channel_slice(np.arange(0,canal_lim))
else:
    rec_2_correct = recording_layers["cmr"]#.frame_slice(0,100)#.channel_slice(np.arange(0,canal_lim))
display(rec_2_correct)

In [3]:
if True:
    engine="dask"
    GPU_available=False
    if reAnalyse:
        match engine,GPU_available:
            case "dask", True:
                engineParam=dict(
                                    job_extra_directives=['--gres=gpu:1g.20gb:2','--gpus=2'],
                                    queue='GPU',
                                    cores=1,
                                    memory="40GB",
                                    nanny=False,
                                    walltime="01:00:00",
                )
            case "dask", False:
                engineParam=dict(
                                    queue='CPU',
                                    cores=1,
                                    memory="50GB",
                                    job_cpu=55,
                                    nanny=False,
                                    walltime="01:00:00",
                )
            case "submitit", True:
                engineParam=dict(
                                    slurm_array_parallelism=40,
                                    mem_gb=16,
                                    timeout_min=20,
                                    slurm_partition="GPU",
                                    cpus_per_task=2
                )
            case "submitit", False:
                engineParam=dict(
                                    slurm_array_parallelism=4,
                                    mem_gb=60,
                                    timeout_min=20,
                                    slurm_partition="CPU",
                                    cpus_per_task=40
                )
            case _:
                engineParam = dict()

        recording_corrected, motion, motion_info  = await runFunction(engine,check_drift,rec_2_correct, probe, **engineParam)
        recording_layers["corrected"] = recording_corrected
        display(motion)

NameError: name 'runFunction' is not defined

In [ ]:
if False: #reAnalyse and engine=="dask":
    if False: #GPU_available: # takes about 8 minutes
        cluster = SLURMCluster(
                        queue='GPU',
                        cores=1,
                        memory="40GB",
                        #job_cpu=55, #should concider 1
                        walltime="01:00:00",
                        log_directory="si_dask_logs",
                        nanny=False,
                        #n_workers=1, # seems to divide ressources
                        job_extra_directives=['--gres=gpu:1g.20gb:2','--gpus=2'],
                        scheduler_options={
                            "dashboard_address": ":8780",
                                           } #port 8787 already used by jupyter
                        )
    else: # takes about 10 minutes
        cluster = SLURMCluster(
                        queue='CPU',
                        cores=1,
                        memory="40GB",
                        job_cpu=55,
                        walltime="01:00:00",
                        log_directory="si_dask_logs",
                        nanny=False,
                        n_workers=1, # seems to divide ressources
                        scheduler_options={"dashboard_address": ":8780"} #port 8787 already used by jupyter
                        )
    cluster.scale(1)
    client = Client(cluster)

    print(cluster.job_script())

    start_time = time.time()
    future = client.submit(check_drift, rec_2_correct, probe) 
    #TODO : allow drift calculation on a few channels only

    #recording_corrected, motion, motion_info = future.result() 
    temp = future.result() 
    
    print(f"job done in {time.time()- start_time} s")

    # Close cluster
    client.close()
    cluster.close()


    recording_layers["corrected"] = recording_corrected
    display(motion)


In [ ]:
#engine="submitit"
if False: #reAnalyse and engine=="submitit":
    # takes about 10 min without slurm more like 16 min with submitit
    GPU_available = False

    start_time = time.time()

    executor = submitit.AutoExecutor(folder=os.getcwd()+'/si_logs/')
    if GPU_available:
        executor.update_parameters(slurm_array_parallelism=40, mem_gb=16, timeout_min=20, slurm_partition="GPU", cpus_per_task=2)
        #executor.update_parameters(mem_gb=16, timeout_min=20, slurm_partition="GPU", slurm_gres='gpu:1')
        #
    else:
        executor.update_parameters(slurm_array_parallelism=4, mem_gb=60, timeout_min=20, slurm_partition="CPU", cpus_per_task=40)
    job = executor.submit(check_drift, rec_2_correct, probe)

    # print the ID of your job
    print("submit job" + str(job.job_id))  

    # await a single result
    await job.awaitable().results()
    print(f"job {job.job_id} completed in " + str(time.time()-start_time) + " seconds")

    last_job = job
    (recording_corrected, motion, motion_info) = job.result()

    display(motion)

In [ ]:
# (72) 16 min (slurm_array_parallelism=4, mem_gb=60, timeout_min=20, slurm_partition="CPU", cpus_per_task=40)
print("""rq: you should avoid submitting multiple small tasks with submitit, which would create many independent jobs
      and possibly overload the cluster, while you can do it without any problem through dask.distributed.""")

In [ ]:
si.plot_motion_info(motion_info, recording_corrected,
                   color_amplitude=True,
        amplitude_cmap="inferno",)

In [ ]:
if not whitenFirst:
    recording_layers = whiten(recording_corrected,recording_layers,"whitenLast")

In [ ]:
si.plot_traces(recording_layers, backend='ipywidgets') #, mode='line'

In [ ]:
if whitenFirst:
    #rec = recording_layers["whitenFirst"] #if whitened and not corrected
    rec = recording_layers["corrected"] #if whitened then corrected
else:
    rec = recording_layers["whitenLast"] #if corrected the whiten

## Identify spike clusters for the first few minutes of recording

It is good practice to have a look at available ressources and current use of the cluster

In [ ]:
checkRessources()

#!sinfo --nodes=node15 -o "%50N  %10c  %20m  %30G "
!squeue --partition="GPU"

#torch.OutOfMemoryError: CUDA out of memory. Tried to allocate 712.00 MiB. GPU 0 has a total capacity of 79.26 GiB of which 187.19 MiB is free. Process 1619368 has 77.78 GiB memory in use. Including non-PyTorch memory, this process has 1.28 GiB memory in use. Of the allocated memory 529.55 MiB is allocated by PyTorch, and 276.45 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)


In [ ]:

# for information, display a list of all parameters that can be modified for the sorter
params = si.get_default_sorter_params(sorter_name_or_class=sorter)
print(f"For information, parameters that are available for the sorter {sorter} are:\n", params)

In [ ]:
if reAnalyse and engine is None:
    gc.collect()

    start_time = time.time()
    rec_training = recording_corrected.frame_slice(0, 30_000 * 60 * duration_extract)
    sorter_params=dict(
        do_correction = False,
        skip_kilosort_preprocessing = True # we already did it
    )

    
    
    sorting = GenerateDict(rec_training, probe, sorterFolder, **sorter_params)

    display(sorting)

In [ ]:
engine = "dask"
if reAnalyse and engine=="dask":
    gc.collect()

    start_time = time.time()
    rec_training = recording_corrected.frame_slice(0, 30_000 * 60 * duration_extract)
    sorter_params=dict(
        do_correction = False,
        skip_kilosort_preprocessing = True # we already did it
    )

    
    if GPU_available: # takes about 10 minutes with dask #longer?
        cluster = SLURMCluster(
                        queue='GPU',
                        cores=1,
                        memory="4GB",
                        job_cpu=1,
                        walltime="00:10:00",
                        log_directory="si_dask_logs",
                        nanny=False,
                        n_workers=1, # seems to divide ressources
                        #worker_extra_args=["--resources GPU=2"],
                        job_extra_directives=['--gpus=2'],
                        scheduler_options={"dashboard_address": ":8780"} #port 8787 already used by jupyter
                        )
    else: # takes about 40 minutes
       cluster = SLURMCluster(
                        queue='CPU',
                        cores=1,
                        memory="6GB",
                        job_cpu=55,
                        walltime="02:00:00",
                        log_directory="si_dask_logs",
                        nanny=False,
                        n_workers=1, # seems to divide ressources
                        scheduler_options={"dashboard_address": ":8780"} #port 8787 already used by jupyter
                        )
       
    cluster.scale(1)
    client = Client(cluster)
    client.get_versions(check=True)
    
    from dask.distributed import PipInstall
    plugin = PipInstall(packages=["git+https://github.com/SpikeInterface/spikeinterface.git"], pip_options=["--upgrade"])
    client.register_plugin(plugin)
    plugin = PipInstall(packages=["git+https://github.com/SpikeInterface/probeinterface.git"], pip_options=["--upgrade"])
    client.register_plugin(plugin)

    print(cluster.job_script()) 


    start_time = time.time()
    future = client.submit(GenerateDict, rec_training, probe, sorterFolder, **sorter_params)
    sorting = future.result() 
    print(f"job done in {time.time()- start_time} s")

    # Close cluster
    client.close()
    cluster.close()

    display(sorting)

In [ ]:
if reAnalyse and engine=="submitit":
    #it takes about 90s with GPU (if available) ; 40 min otherwise
    gc.collect()

    GPU_available = True

    start_time = time.time()
    rec_training = recording_corrected.frame_slice(0, 30_000 * 60 * duration_extract)

    executor = submitit.AutoExecutor(folder=os.getcwd()+'/si_logs/')
    if GPU_available:
        executor.update_parameters(mem_gb=5, timeout_min=10, slurm_partition="GPU", cpus_per_task=2)
        #executor.update_parameters(mem_gb=5, timeout_min=5, slurm_partition="GPU", slurm_gres='gpu:1')
    else:
        executor.update_parameters(mem_gb=5, timeout_min=120, slurm_partition="CPU", cpus_per_task=60)


    # actually submit the job
    job = executor.submit(GenerateDict, rec_training, probe)

    # print the ID of your job
    print("submit job" + str(job.job_id))  

    # await a single result
    await job.awaitable().results()
    print(f"job {job.job_id} completed in " + str(time.time()-start_time) + " seconds")

    last_job = job
    sorting = job.result()
    display(sorting)

In [ ]:
#kilosort4 run time 2212.34s for 8Gb 10cpus num1 (15.74, 15.42, 1.49, 2.11)
#100%|██████████| 60/60 [39:44<00:00, 39.74s/it] 8/10 python

#32%|███▏      | 19/60 [09:26<19:40, 28.78s/it] 8/30 submitit
#32%|███▏      | 19/60 [09:13<19:55, 29.16s/it] 16/30 submitit
#60%|██████    | 36/60 [12:10<07:36, 19.02s/it] 5/30 submitit
#23%|██▎       | 14/60 [10:14<32:55, 42.95s/it] 5/10 submitit
#42%|████▏     | 25/60 [06:35<07:23, 12.66s/it] 5/50 submitit
#33%|███▎      | 20/60 [05:32<10:29, 15.75s/it] 5/50 submitit data in mnt


#GPU
#job completed: 33972 returned in 110.73936009407043 seconds 5/50
#job completed: 33973 returned in 94.93399000167847 seconds 5/10
#job completed: 33975 returned in 93.79518842697144 seconds 5/2

In [ ]:
if not reAnalyse:
    if os.path.isdir(sorterFolder):
        # directory exists
        print(f"the previous folder {sorterFolder} was found, importing the data")
        sorting = si.read_sorter_folder(sorterFolder)
        display(sorting)
    else:
        print(f"the folder {sorterFolder} does not exist ; make sure of the path or reAnalyse the data")


## Cure the clusters
Here you should ensure that you are happy with the clusters that were found. For that, you should first compute analysis for the training clusters

### Fast initial curation

In [ ]:
display(sorting)
sorting = si.remove_duplicated_spikes(sorting=sorting)
sorting = si.remove_excess_spikes(sorting=sorting, recording=recording_corrected)
#Est-ce qu'on veut aussi remove_redundant_units?
display(sorting)

### Construct analyzer


In [ ]:
if reAnalyse and engine is None:
    
    start_time = time.time()
    compute_analyzer(sorting, rec_training, probe, training_folder) #, append=True) # uncomment to redo only a few analysis

    #display(sorting_analyzer_training)

In [ ]:
reAnalyse=True
GPU_available=False
if reAnalyse and engine=="dask":
    gc.collect()
    
    if GPU_available: # takes about 1.5 minutes with dask
        cluster = SLURMCluster(
                        queue='GPU',
                        cores=1,
                        memory="70GB",
                        job_cpu=70,
                        walltime="01:20:00",
                        log_directory="si_dask_logs",
                        nanny=False,
                        n_workers=40, # seems to divide ressources
                        #worker_extra_args=["--resources GPU=2"],
                        job_extra_directives=['--gpus=2'],
                        scheduler_options={"dashboard_address": ":8780"} #port 8787 already used by jupyter
                        )
    else: # takes about 1.5 min
       cluster = SLURMCluster(
                        queue='CPU',
                        cores=1,
                        memory="60GB",
                        job_cpu=40,
                        walltime="02:00:00",
                        log_directory="si_dask_logs",
                        nanny=False,
                        n_workers=40, # seems to divide ressources
                        scheduler_options={"dashboard_address": ":8780"} #port 8787 already used by jupyter
                        )
       
    cluster.scale(1)
    client = Client(cluster)

    print(cluster.job_script()) 


    start_time = time.time()
    future = client.submit(compute_analyzer, sorting, rec_training, probe, training_folder) #, append=True) # uncomment to redo only a few analysis
    future.result()
    #sorting_analyzer_training = future.results() 
    print(f"job done in {time.time()- start_time} s")

    # Close cluster
    client.close()
    cluster.close()

    #display(sorting_analyzer_training)

Now, you have 2 options:
- Either go back to local computer for full benefice of spikeinterface_gui
1. First, copy the sorting_analyzer_training folder to crnldata ([see next cell](#download))
1. Then, go on a local (not over ssh) script at [the most interactive viewing part](#local) at the end of this notebook, reload the sorting_analyzer older, and visualize everything on a gui.
1. Finally, when you are happy with the spike clusters, you can upload it back to the crnl cluster to proceed with [full sorting](#sort-full-recording)
- Or use [the following embedded plotting widgets](#widgets)

### Option1: go back to local PC
<a id='download'></a>

In [ ]:
# copy initial sorting to crnldata for viewing
if reAnalyse and engine=="dask":
    #it takes about 10s

    # takes about 10 minutes with dask
    cluster = SLURMCluster(cores=1,
                        memory="1GB",
                        job_cpu=1,
                        walltime="00:05:00",
                        log_directory="si_dask_logs",
                        nanny=False,
                        n_workers=1, # seems to divide ressources
                        scheduler_options={"dashboard_address": ":8780"} #port 8787 already used by jupyter
                        )
       
    cluster.scale(1)
    client = Client(cluster)

    src=training_folder
    currentFile_data = magicretrieve('currentFile_data')
    dst=os.path.join(os.path.split(currentFile_data)[0],src)
    start_time = time.time()
    future = client.submit(shutil.copytree, src, dst) 
    _ = future.result() 
    print(f"job done in {time.time()- start_time} s")

    # Close cluster
    client.close()
    cluster.close()

    display(sorting)

**Here is when you should work locally**
and do the last part

... and then come back on ssh 

In [ ]:
#import back sorting_analyzer
# reAnalyse = True
if reAnalyse and engine=="dask":
    #it takes about 10s

    # takes about 10 minutes with dask
    cluster = SLURMCluster(cores=1,
                        memory="1GB",
                        job_cpu=1,
                        walltime="00:05:00",
                        log_directory="si_dask_logs",
                        nanny=False,
                        n_workers=1, # seems to divide ressources
                        scheduler_options={"dashboard_address": ":8780"} #port 8787 already used by jupyter
                        )
       
    cluster.scale(1)
    client = Client(cluster)

    dst=training_folder
    currentFile_data = magicretrieve('currentFile_data')
    src=os.path.join(os.path.split(currentFile_data)[0],dst)
    start_time = time.time()
    future = client.submit(shutil.copytree, src, dst) 
    _ = future.result() 
    print(f"job done in {time.time()- start_time} s")

    # Close cluster
    client.close()
    cluster.close()

    display(sorting)

In [ ]:
client.close()
cluster.close()

### Option2: inline visualisation
<a id='widgets'></a>

In [ ]:
sorting_analyzer_training = si.load_sorting_analyzer("sorting_analyzer_training")
display(sorting_analyzer_training)

In [ ]:
# select a few ids (unités)
#unit_ids=[1, 2, 5]
unit_ids=sorting_analyzer_training.unit_ids

In [ ]:
si.plot_unit_templates(sorting_analyzer_training, unit_ids=unit_ids, backend="ipywidgets")

In [ ]:
# select a few ids
unit_ids=[1, 2, 5]

In [ ]:
si.plot_rasters(sorting_analyzer_training, unit_ids=unit_ids)

In [ ]:
try:
    si.plot_isi_distribution(sorting_analyzer_training,unit_ids=unit_ids)
except Exception as error:
    print(error)

In [ ]:
si.plot_autocorrelograms(sorting_analyzer_training, unit_ids=unit_ids)

In [ ]:
si.plot_crosscorrelograms(sorting_analyzer_training, unit_ids=unit_ids)

In [ ]:
si.plot_unit_presence(sorting_analyzer_training)

## Sort full recording

In [ ]:
if reAnalyse and engine=="dask":
    gc.collect()
    
    if GPU_available: # takes about 10 minutes with dask
        cluster = SLURMCluster(
                        queue='GPU',
                        cores=1,
                        memory="70GB",
                        job_cpu=70,
                        walltime="00:20:00",
                        log_directory="si_dask_logs",
                        nanny=False,
                        n_workers=40, # seems to divide ressources
                        #worker_extra_args=["--resources GPU=2"],
                        job_extra_directives=['--gpus=2'],
                        scheduler_options={"dashboard_address": ":8780"} #port 8787 already used by jupyter
                        )
    else: # takes about 10 minutes
       cluster = SLURMCluster(
                        queue='CPU',
                        cores=1,
                        memory="6GB",
                        job_cpu=55,
                        walltime="02:00:00",
                        log_directory="si_dask_logs",
                        nanny=False,
                        n_workers=1, # seems to divide ressources
                        scheduler_options={"dashboard_address": ":8780"} #port 8787 already used by jupyter
                        )
       
    cluster.scale(1)
    client = Client(cluster)

    print(cluster.job_script()) 


    start_time = time.time()
    future = client.submit(compute_analyzer, sorting, recording_corrected, probe, fullAnalyzer_folder)
    future.result()
    #sorting_analyzer_training = future.results() 
    print(f"job done in {time.time()- start_time} s")

    # Close cluster
    client.close()
    cluster.close()

    #display(sorting_analyzer_training)

In [ ]:
if reAnalyse and engine=='submitit':
    start_time = time.time()

    executor = submitit.AutoExecutor(folder=os.getcwd()+'/si_logs/')
    #executor.update_parameters(slurm_array_parallelism=2, mem_gb=30, timeout_min=10, slurm_partition="CPU", cpus_per_task=50)
    executor.update_parameters(mem_gb=60, timeout_min=20, slurm_partition="GPU", cpus_per_task=70) #cpus_per_task

    # actually submit the job
    job = executor.submit(compute_analyzer, sorting, recording_corrected, probe, fullAnalyzer_folder)

    # print the ID of your job
    print("submit job" + str(job.job_id))  

    # await a single result
    await job.awaitable().results()
    print(f"job {job.job_id} completed in " + str(time.time()-start_time) + " seconds")

In [ ]:
#job 34074 completed in 408.1813361644745 second (slurm_array_parallelism=2, mem_gb=60, timeout_min=20, slurm_partition="GPU", cpus_per_task=50)
#job 34078 completed in 437.409494638443 seconds (slurm_array_parallelism=3, mem_gb=60, timeout_min=20, slurm_partition="GPU", cpus_per_task=50)
#job 34081 completed in 423.37460565567017 seconds (slurm_array_parallelism=2, mem_gb=60, timeout_min=20, slurm_partition="GPU", slurm_gres="gpu:2", cpus_per_task=50)
#job 34085 completed in 367.0000305175781 seconds (slurm_array_parallelism=2, mem_gb=60, timeout_min=20, slurm_partition="GPU", cpus_per_task=70)
#job 34089 completed in 370.1982145309448 seconds (slurm_array_parallelism=2, mem_gb=60, timeout_min=20, slurm_partition="GPU", cpus_per_task=80)
#job 34093 completed in 355.1876621246338 seconds (mem_gb=60, timeout_min=20, slurm_partition="GPU", cpus_per_task=70)

last_job = job
checkRessources()

In [ ]:
if not reAnalyse:
    print("not running analysis but loading previous one")
    sorting_analyzer = si.load_sorting_analyzer(fullAnalyzer_folder)
    display(sorting_analyzer)

In [ ]:
si.export_report(sorting_analyzer=sorting_analyzer,output_folder='report')

# Most interactive viewing on local
<a id='local'></a>

In [ ]:
#takes 13s
#TODO: here should find a way to retrieve magicdata from ssh
baseName = '/crnldata/waking/audrey_hay/NPX/NPX1/VB/Expe_2024-07-22_17-55-16/'
sorting_analyzer_training = si.load_sorting_analyzer(os.path.join('//10.69.168.1',baseName,training_folder))
display(sorting_analyzer_training)

In [ ]:
%gui qt
si.plot_sorting_summary(sorting_analyzer_training, backend="spikeinterface_gui")

In [ ]:
client.close()
cluster.close()